<a href="https://colab.research.google.com/github/githrajesh/source/blob/master/Course_NVIDIA_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Course NVIDIA - Homework

> Dataset: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents

In [ ]:
!pip install -q kaggle

* Inside the Kaggle dashboard, go to the Settings page
https://www.kaggle.com/settings

* In the API section, select the button [Create New Token]

* A file called kaggle.json will be downloaded

* Send this file to Colab.


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                                title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/apple-quality                                     Apple Quality                                      170KB  2024-01-11 14:31:07           6066        144  1.0              
hummaamqaasim/jobs-in-data                                         Jobs and Salaries in Data Science                   76KB  2023-12-25 20:03:32          12412        228  1.0              
emirhanai/city-happiness-index-2024                                City Happiness Index - 2024                          8KB  2024-01-22 00:41:50           1453         32  1.0              
shiivvvaam/revenue-of-top-companies-in-india      

In [ ]:
!kaggle datasets download -d sobhanmoosavi/us-accidents

In [ ]:
!mkdir dataset

In [ ]:
!unzip us-accidents.zip -d dataset

## Installing and importing the libraries

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 438, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 438 (delta 121), reused 57 (delta 54), pack-reused 269
Receiving objects: 100% (438/438), 118.31 KiB | 2.32 MiB/s, done.
Resolving deltas: 100% (223/223), done.
***********************************************************************
We will now install RAPIDS via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.2/955.2 MB 723.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 842.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.8/142.8 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import cudf
import cuml
import cupy as cp

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 1) Reading the dataset

In [ ]:
%time
dataset = cudf.read_csv('/content/dataset/US_Accidents_March23.csv')

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.44 µs


In [ ]:
dataset.shape

(7728394, 46)

## 2) Removing missing data

In [ ]:
df = dataset.dropna()

In [ ]:
df

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
3402762,A-3412645,Source1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,...,False,False,False,False,False,False,Night,Night,Night,Night
3402767,A-3412650,Source1,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,...,False,False,False,False,False,False,Day,Day,Day,Day
3402771,A-3412654,Source1,2,2016-02-08 11:51:46,2016-02-08 17:51:46,41.375310,-81.820170,41.367860,-81.821740,0.521,...,False,False,False,False,False,False,Day,Day,Day,Day
3402773,A-3412656,Source1,2,2016-02-08 15:16:43,2016-02-08 21:16:43,40.109310,-82.968490,40.110780,-82.984000,0.826,...,False,False,False,False,False,False,Day,Day,Day,Day
3402774,A-3412657,Source1,2,2016-02-08 15:43:50,2016-02-08 21:43:50,39.192880,-84.477230,39.196150,-84.473350,0.307,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728389,A-7777757,Source1,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379360,33.998880,-117.370940,0.543,...,False,False,False,False,False,False,Day,Day,Day,Day
7728390,A-7777758,Source1,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148060,32.765550,-117.153630,0.338,...,False,False,False,False,False,False,Day,Day,Day,Day
7728391,A-7777759,Source1,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847790,33.777400,-117.857270,0.561,...,False,False,False,False,False,False,Day,Day,Day,Day
7728392,A-7777760,Source1,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.992460,-118.403020,33.983110,-118.395650,0.772,...,False,False,False,False,False,False,Day,Day,Day,Day


In [ ]:
type(df)

cudf.core.dataframe.DataFrame

## 3) Removing columns

In [ ]:
cols_dataset = df.columns.values.tolist()
cols_dataset

['ID',
 'Source',
 'Severity',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'End_Lat',
 'End_Lng',
 'Distance(mi)',
 'Description',
 'Street',
 'City',
 'County',
 'State',
 'Zipcode',
 'Country',
 'Timezone',
 'Airport_Code',
 'Weather_Timestamp',
 'Temperature(F)',
 'Wind_Chill(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Wind_Direction',
 'Wind_Speed(mph)',
 'Precipitation(in)',
 'Weather_Condition',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Turning_Loop',
 'Sunrise_Sunset',
 'Civil_Twilight',
 'Nautical_Twilight',
 'Astronomical_Twilight']

In [ ]:
cols = ['Severity', 'Source', 'County', 'State', 'Weather_Condition', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']
cols

['Severity',
 'Source',
 'County',
 'State',
 'Weather_Condition',
 'Sunrise_Sunset',
 'Civil_Twilight',
 'Nautical_Twilight',
 'Astronomical_Twilight',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Turning_Loop']

In [ ]:
len(cols)

22

In [ ]:
drop_cols = [c for c in cols_dataset if c not in cols]
drop_cols

['ID',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'End_Lat',
 'End_Lng',
 'Distance(mi)',
 'Description',
 'Street',
 'City',
 'Zipcode',
 'Country',
 'Timezone',
 'Airport_Code',
 'Weather_Timestamp',
 'Temperature(F)',
 'Wind_Chill(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Wind_Direction',
 'Wind_Speed(mph)',
 'Precipitation(in)']

In [ ]:
df = df.drop(columns=drop_cols)

In [ ]:
df

,Source,Severity,County,State,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
3402762,Source1,3,Franklin,OH,Light Rain,False,False,False,False,False,...,False,False,False,False,False,False,Night,Night,Night,Night
3402767,Source1,3,Hamilton,OH,Light Rain,False,False,False,False,False,...,False,False,False,False,False,False,Day,Day,Day,Day
3402771,Source1,2,Cuyahoga,OH,Snow,False,False,False,False,True,...,False,False,False,False,False,False,Day,Day,Day,Day
3402773,Source1,2,Franklin,OH,Snow,False,False,False,False,False,...,False,False,False,False,False,False,Day,Day,Day,Day
3402774,Source1,2,Hamilton,OH,Light Snow,False,False,False,False,False,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728389,Source1,2,Riverside,CA,Fair,False,False,False,False,False,...,False,False,False,False,False,False,Day,Day,Day,Day
7728390,Source1,2,San Diego,CA,Fair,False,False,False,False,False,...,False,False,False,False,False,False,Day,Day,Day,Day
7728391,Source1,2,Orange,CA,Partly Cloudy,False,False,False,False,True,...,False,False,False,False,False,False,Day,Day,Day,Day
7728392,Source1,2,Los Angeles,CA,Fair,False,False,False,False,False,...,False,False,False,False,False,False,Day,Day,Day,Day


## 4) Label encoder



In [ ]:
from cuml.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()

```
df['Weather_Condition'] = label_encoder.fit_transform(df['Weather_Condition'])
df['Amenity']= label_encoder.fit_transform(df['Amenity'])
[...]
```

In [ ]:
for c in cols:
  if c != 'Severity':
    df[c] = label_encoder.fit_transform(df[c])

In [ ]:
df

,Source,Severity,County,State,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
3402762,0,3,532,33,53,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
3402767,0,3,638,33,53,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3402771,0,2,388,33,97,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3402773,0,2,532,33,97,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3402774,0,2,638,33,61,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728389,0,2,1258,3,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7728390,0,2,1310,3,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7728391,0,2,1098,3,76,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7728392,0,2,870,3,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 5) Balancing



In [ ]:
df['Severity'].value_counts()

2    3348445
4     112511
3      68026
1      25567
Name: Severity, dtype: int64

In [ ]:
df = df.to_pandas()

In [ ]:
type(df)

pandas.core.frame.DataFrame

In [ ]:
from sklearn.utils import resample

In [ ]:
df_s1 = df[df['Severity'] == 1]
df_s2 = df[df['Severity'] == 2]
df_s3 = df[df['Severity'] == 3]
df_s4 = df[df['Severity'] == 4]

In [ ]:
df_s1

,Source,Severity,County,State,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
6525681,0,1,1163,2,71,0,0,1,1,0,...,0,0,1,0,0,0,0,0,0,0
6527285,0,1,1163,2,14,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
6540036,0,1,868,33,58,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6542183,0,1,912,2,14,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
6543739,0,1,1384,3,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7228848,0,1,1571,25,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7228849,0,1,200,25,76,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7228851,0,1,209,25,71,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7228852,0,1,1571,25,71,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_s1.count()[0], df_s2.count()[0], df_s3.count()[0], df_s4.count()[0]

(25567, 3348445, 68026, 112511)

In [ ]:
count = int(max(df_s1.count()[0], df_s2.count()[0], df_s3.count()[0], df_s4.count()[0]) / 60)

In [ ]:
count

55807

In [ ]:
3348445 / 20

167422.25

In [ ]:
df_s1 = resample(df_s1, replace=df_s1.count()[0] < count, n_samples=count, random_state=42)
df_s2 = resample(df_s2, replace=df_s2.count()[0] < count, n_samples=count, random_state=42)
df_s3 = resample(df_s3, replace=df_s3.count()[0] < count, n_samples=count, random_state=42)
df_s4 = resample(df_s4, replace=df_s4.count()[0] < count, n_samples=count, random_state=42)

In [ ]:
df = pd.concat([df_s1, df_s2, df_s3, df_s4])

In [ ]:
print(df['Severity'].value_counts())

#df.groupby(by='Severity')['Severity'].count()

1    55807
2    55807
3    55807
4    55807
Name: Severity, dtype: int64


## 6) OneHot Encoding

In [ ]:
X = df.drop('Severity', axis=1)

In [ ]:
X

,Source,County,State,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
7144412,0,465,16,76,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7087188,0,851,8,7,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6935720,0,1430,3,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6973530,0,912,2,14,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,0,0
7129626,0,912,2,14,0,0,1,0,0,0,...,0,0,0,0,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492362,0,494,8,14,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
7164039,0,1098,3,71,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5612443,0,338,8,14,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7027731,0,532,33,76,0,0,0,0,0,0,...,0,1,0,0,1,0,1,1,0,0


In [ ]:
from cuml.preprocessing import OneHotEncoder
from cuml.compose import ColumnTransformer

In [ ]:
idx = [*range(0,20)]
idx

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [ ]:
onehotencoder = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(sparse=False), idx)], remainder='passthrough')

In [ ]:
X = onehotencoder.fit_transform(X)

In [ ]:
X.shape

(223228, 1688)

## 7) Standardization

In [ ]:
from cuml.preprocessing import StandardScaler
scaler_dataset = StandardScaler()

X = scaler_dataset.fit_transform(X)

In [ ]:
print(X)
print(X.shape)

        0        1        2         3         4        5         6     \
0        0.0 -0.00898 -0.00702 -0.015982 -0.020636 -0.00702 -0.080937   
1        0.0 -0.00898 -0.00702 -0.015982 -0.020636 -0.00702 -0.080937   
2        0.0 -0.00898 -0.00702 -0.015982 -0.020636 -0.00702 -0.080937   
3        0.0 -0.00898 -0.00702 -0.015982 -0.020636 -0.00702 -0.080937   
4        0.0 -0.00898 -0.00702 -0.015982 -0.020636 -0.00702 -0.080937   
...      ...      ...      ...       ...       ...      ...       ...   
223223   0.0 -0.00898 -0.00702 -0.015982 -0.020636 -0.00702 -0.080937   
223224   0.0 -0.00898 -0.00702 -0.015982 -0.020636 -0.00702 -0.080937   
223225   0.0 -0.00898 -0.00702 -0.015982 -0.020636 -0.00702 -0.080937   
223226   0.0 -0.00898 -0.00702 -0.015982 -0.020636 -0.00702 -0.080937   
223227   0.0 -0.00898 -0.00702 -0.015982 -0.020636 -0.00702 -0.080937   

            7         8       9     ...      1678      1679  1680      1681  \
0      -0.002993 -0.022406 -0.0056  ...  0.5

## 8) Creating the variables

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = df['Severity']

y = LabelEncoder().fit_transform(y)

In [ ]:
y

7144412    0
7087188    0
6935720    0
6973530    0
7129626    0
          ..
6492362    3
7164039    3
5612443    3
7027731    3
7248793    3
Length: 223228, dtype: uint8

In [ ]:
y.shape

(223228,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((189743, 1688), (33485, 1688), (189743,), (33485,))

In [ ]:
import pickle
with open('dataset.pkl', mode = 'wb') as f:
  pickle.dump([X_train, y_train, X_test, y_test], f)

## 9) Training

In [ ]:
from cuml.neighbors import KNeighborsClassifier

In [ ]:
with open('dataset.pkl', 'rb') as f:
  X_train, y_train, X_test, y_test = pickle.load(f)

In [ ]:
X_train_cudf = cudf.DataFrame.from_pandas(X_train)
X_test_cudf = cudf.DataFrame.from_pandas(X_test)

y_train_cudf = cudf.Series(y_train.values)
y_test_cudf = cudf.Series(y_test.values)

In [ ]:
type(X_train_cudf), type(X_test_cudf)

(cudf.core.dataframe.DataFrame, cudf.core.dataframe.DataFrame)

In [ ]:
type(y_train_cudf), type(y_test_cudf)

(cudf.core.series.Series, cudf.core.series.Series)

In [ ]:
%time

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train_cudf, y_train_cudf)

CPU times: user 0 ns, sys: 4 µs, total: 4 µs
Wall time: 6.91 µs


KNeighborsClassifier()

## 10) Predictions and evaluation

In [ ]:
predictions = knn.predict(X_test_cudf)
predictions

112649    2
136161    1
195733    3
128522    2
158793    3
         ..
193027    1
222648    3
55405     0
139854    3
83729     3
Length: 33485, dtype: uint8

In [ ]:
from cuml.metrics import accuracy_score

In [ ]:
accuracy_score(y_test_cudf, predictions)

0.6250858306884766

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!nvidia-smi

Wed Jan 31 14:33:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0              33W /  70W |   7877MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--